In [97]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import implicit
import scipy

df1 = pd.read_excel("./drugnsc.xlsx")
df2 = pd.read_excel("./nscquerylist.xlsx")

In [98]:
df1.head()

,NSC # b,Drug name,FDA status,Mechanism of action c,PubChem SID,SMILES d,BR:MCF7,BR:MDA-MB-231,BR:HS 578T,BR:BT-549,...,RE:786-0,RE:A498,RE:ACHN,RE:CAKI-1,RE:RXF 393,RE:SN12C,RE:TK-10,RE:UO-31,Total experiments,Total after quality control
0,1,tolylquinone,-,-,66954,CC1=CC(=O)C=CC1=O,-0.27,-0.3,-0.82,-0.23,...,-0.52,-1.65,1.66,-0.27,0,-0.39,-0.38,1.06,3,2
1,17,4-amino-3-pentadecylphenol,-,-,66970,CCCCCCCCCCCCCCCc1cc(O)ccc1N,-0.35,-0.3,-0.22,1.48,...,0.52,-0.33,-0.94,0.72,-0.24,-0.32,-0.76,1.12,3,3
2,89,(dimethylamino)propiophenone hydrochloride,-,-,538184,[Cl-].CN(C)CCC(=O)c1ccccc1,na,na,na,na,...,0.37,-0.66,-0.14,0.74,-0.44,-0.18,-0.1,-0.12,3,2
3,185,Cactinomycin,-,-,67121,C[C@H]1C[C@H](C)C(=O)[C@@H](C1)[C@H](O)CC2CC(=...,na,na,na,na,...,-0.06,na,na,1.68,0.92,1.39,-0.9,0.92,2,1
4,295,2-Phenylbutyric Acid,-,-,67217,OC(=O)CCCc1ccccc1,-0.26,-0.26,-0.26,-0.26,...,4.82,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,-0.26,2,1


In [111]:
copy_df1 = df1.drop('Drug name', 1).drop('FDA status', 1).drop('Mechanism of action c', 1).drop('PubChem SID', 1).drop('SMILES d', 1).drop('Total experiments', 1).drop('Total after quality control', 1)


In [112]:
# copy_df1 = copy_df1.set_index('NSC # b')
copy_df1 = copy_df1.set_index('NSC # b')
copy_df1 = copy_df1.replace('na', '0.00')
copy_df1 = copy_df1.stack().reset_index().rename(columns={'level_0':'drug','level_1':'cancer_cell_lines', 0:'z_score'})

In [117]:
copy_df1.sample(3)

,NSC # b,cancer_cell_lines,z_score
223057,338519,LC:HOP-92,0.44
903710,704345,PR:PC-3,-0.35
1110454,732787,LC:A549/ATCC,-0.54


In [119]:
copy_df1['NSC # b'] = copy_df1['NSC # b'].astype("category")
copy_df1['cancer_cell_lines'] = copy_df1['cancer_cell_lines'].astype("category")

In [137]:
copy_df1.head()

,NSC # b,cancer_cell_lines,z_score
0,1,BR:MCF7,-0.27
1,1,BR:MDA-MB-231,-0.3
2,1,BR:HS 578T,-0.82
3,1,BR:BT-549,-0.23
4,1,BR:T-47D,1.93


In [150]:
z_scores = scipy.sparse.coo_matrix((copy_df1['z_score'].astype(float),
                       (copy_df1['cancer_cell_lines'].cat.codes.copy(),
                        copy_df1['NSC # b'].cat.codes.copy())))

z_scores # cell line x drugsdrug

<60x21738 sparse matrix of type '<class 'numpy.float64'>'
	with 1304280 stored elements in COOrdinate format>

In [153]:
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
z_scores = bm25_weight(z_scores, K1=100, B=0.8)
model = implicit.als.AlternatingLeastSquares(factors=50)
model.fit(z_scores)

# recommend drug for cell line
drugs_cell_lines = z_scores.T.tocsr()
# 2 is index, can substitute any drug from list
recommendation = model.recommend(20000, drugs_cell_lines)

# find related cell lines
# 2 is index, can sub
related = model.similar_items(50)

In [155]:
related

[(50, 0.99999999999999989),
 (40, 0.99995350686145745),
 (35, 0.9998442924576243),
 (45, 0.99978309388734088),
 (42, 0.99977761025848189),
 (5, 0.99973647923906273),
 (9, 0.99968274618596731),
 (57, 0.99967291075395837),
 (36, 0.99962458464811932),
 (48, 0.99956906959258596)]